In [59]:
from src.game_model import GameModel, MAX_ENVIRONMENT_SIZE_TO_PRINT

## Declare environment and agents:

In [60]:
"""
Initializes the GameModel with a labeled environment.

Args:
    default_label (list of str): The default labels to assign to each column in the environment.
    additional_labels (list of str): Other possible labels that can be assigned to the environment elements.

    default_agents_labels (list of str)): The default labels to each row of agent features.
    additional_agents_labels (list of list of strings): For each agent feature, specify other possible labels that can be assigned to each.

    environment_dimensions (list of int): Dimensions of the environment matrix. 
    Consider adding an additional dimension to the environment for time if rules refer to previous timesteps.

    agents_dimensions (list of int): Dimensions of the agents matrix.
    Consider to use the first dimension for the number of player and the second dimension for the number of features.

    dimensions_descriptions (str): String explaining each dimension of the environment and of the agents (optional)
    game_name (str): String with the name of the game (optional)
"""

gm = GameModel(
default_labels=['free'], additional_labels=['X', 'O'], environment_dimensions=[3, 3], 
default_agents_labels=['not starter', 'X'], additional_agents_labels=[['starter'], ['O']], agents_dimensions=[2, 2], 
dimensions_descriptions="For the environment we have the 3x3 board and 20 timesteps. For the agents we have 2 players with feature 1 indicating who is playing, waiting and won or lost, and feature 2 indicating their symbol.",
game_name="tic-tac-toe")

print(gm)
print()
gm.print_rules()


        You can use the method gm.action_is_violation_if(rule, rule_description) to express rules for the game.

        Use help(gm.action_is_violation_if) for help on how to define rules.
        The rule: "Nothing is allowed if the game is ended." is defined by default, to delete it use gm.delete_rule(1).
        Use gm.print_rules() to see all the rules that have been setted.
        
        Use gm.set_endgame(callable_function(game)) to define based on what dynamics your game should end.
        The callable function should have one only input parameter, which represents the game itself, 
        which can be used to refer to anything inside its model, this include:
                - environment = the environment or board (e.g. game.environment)
                - agents = the agents involved in the game
                - actions = previous actions performed
                - started = if the game has started
                - ended = if the game has ended
        Example: 
     

## Initialize parameters

In [61]:
gm.agents[1, 1] = 'O'
gm.agents[1, 0] = 'starter'
print(gm)

Game: tic-tac-toe

Number of agents: 2, Number of agent features: 2 
Agent features labels: [['not starter', 'starter'], ['X', 'O']])

Environment shape: (3, 3), Total number of elements: 9 
Labels: ['free', 'X', 'O'])

Dimensions Descriptions: For the environment we have the 3x3 board and 20 timesteps. For the agents we have 2 players with feature 1 indicating who is playing, waiting and won or lost, and feature 2 indicating their symbol.

Agents: 
[['not starter' 'X']
 ['starter' 'O']]

Environment:
[['free' 'free' 'free']
 ['free' 'free' 'free']
 ['free' 'free' 'free']]


## Define the dynamic to end the game

In [62]:
def tic_tac_toe_endgame(game):
    board = game.environment
    # Check rows for winning condition
    for row in board:
        if row[0] == row[1] == row[2] != 'free':
            return True

    # Check columns for winning condition
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] != 'free':
            return True

    # Check diagonals for winning condition
    if board[0][0] == board[1][1] == board[2][2] != 'free':
        return True
    if board[0][2] == board[1][1] == board[2][0] != 'free':
        return True

    # If no winner, return False
    return False

gm.set_endgame(tic_tac_toe_endgame)

## Define the rules

In [63]:
"""Only the player whose starter can play the first turn:
agents[who] = the agent who did the action
agent[0] refers to the agent's first feature, which we assigned to the status
"""
gm.action_is_violation_if(lambda who, where, what, which_agent_feature, game: not game.started and game.agents[who][0] != 'starter', "This is not the starting player and this is the first turn.")

In [64]:
"""Agents need to alternate actions:
The last action was done by the same player.
"""
gm.action_is_violation_if(lambda who, where, what, which_agent_feature, game: game.started and who == game.actions[-1]['who'], "Players cannot play two times consecutively")

In [65]:
"""You can only put a sign if the space is free:
"""
gm.action_is_violation_if(lambda who, where, what, which_agent_feature, game: game.environment[where] != 'free', "The space needs to be free to put a sign on it.")

In [66]:
"""Agents cannot modify their features or the ones of other agents:
"""
gm.action_is_violation_if(lambda who, where, what, which_agent_feature, game: which_agent_feature != [None], "Agents cannot modify their own features.")

In [67]:
"""Agents can only put their own sign:
agent[1] refers to the agent's symbol.
"""
gm.action_is_violation_if(lambda who, where, what, which_agent_feature, game: game.agents[who][1] != what, "Agents can only put their own sign.")

In [68]:
gm.print_rules()

Rules:
1: Nothing is allowed if the game is ended.
2: This is not the starting player and this is the first turn.
3: Players cannot play two times consecutively
4: The space needs to be free to put a sign on it.
5: Agents cannot modify their own features.
6: Agents can only put their own sign.


## Example Actions:

In [69]:
gm.environment

GameMatrix([['free', 'free', 'free'],
            ['free', 'free', 'free'],
            ['free', 'free', 'free']], dtype='<U4')

Action not permitted:

In [70]:
gm.action_on_environment(0, [0, 0], 'X')
gm.environment

Broke rule 2: This is not the starting player and this is the first turn.
This action is not permitted.



GameMatrix([['free', 'free', 'free'],
            ['free', 'free', 'free'],
            ['free', 'free', 'free']], dtype='<U4')

Action that respects the rules:

In [71]:
gm.action_on_environment(1, [0, 0], 'O')
gm.environment

GameMatrix([['O', 'free', 'free'],
            ['free', 'free', 'free'],
            ['free', 'free', 'free']], dtype='<U4')

Other actions that do not respect the rules:

In [72]:
gm.action_on_environment(1, [1, 0], 'O')
gm.environment

Broke rule 3: Players cannot play two times consecutively
This action is not permitted.



GameMatrix([['O', 'free', 'free'],
            ['free', 'free', 'free'],
            ['free', 'free', 'free']], dtype='<U4')

In [73]:
gm.action_on_environment(0, [0, 0], 'X')
gm.environment

Broke rule 4: The space needs to be free to put a sign on it.
This action is not permitted.



GameMatrix([['O', 'free', 'free'],
            ['free', 'free', 'free'],
            ['free', 'free', 'free']], dtype='<U4')

In [74]:
gm.action_on_agent(0, 'O', [0,0])
gm.environment

Broke rule 5: Agents cannot modify their own features.
This action is not permitted.


GameMatrix([['O', 'free', 'free'],
            ['free', 'free', 'free'],
            ['free', 'free', 'free']], dtype='<U4')

In [75]:
gm.action_on_environment(0, [2, 1], 'O')
gm.environment

Broke rule 6: Agents can only put their own sign.
This action is not permitted.



GameMatrix([['O', 'free', 'free'],
            ['free', 'free', 'free'],
            ['free', 'free', 'free']], dtype='<U4')

## Example of game:

It's suggested to make a wrapper definition for actions in the game:

In [76]:
def put_sign(player, coordinates):
    if player == 0:
        sign = 'X'
    elif player == 1:
        sign = 'O'
    else:
        raise ValueError("Player variable has to be 0 or 1")

    performed = gm.action_on_environment(player, coordinates, sign)

    if performed:
        print(gm.environment)

In [77]:
put_sign(0, [1,1])

[['O' 'free' 'free']
 ['free' 'X' 'free']
 ['free' 'free' 'free']]


In [78]:
put_sign(0, [0,1])

Broke rule 3: Players cannot play two times consecutively
This action is not permitted.



In [79]:
put_sign(1, [2,2])

[['O' 'free' 'free']
 ['free' 'X' 'free']
 ['free' 'free' 'O']]


In [80]:
put_sign(0, [2,0])

[['O' 'free' 'free']
 ['free' 'X' 'free']
 ['X' 'free' 'O']]


In [81]:
put_sign(1, [0,2])

[['O' 'free' 'O']
 ['free' 'X' 'free']
 ['X' 'free' 'O']]


In [82]:
gm.ended

False

In [83]:
put_sign(0, [0,0])

Broke rule 4: The space needs to be free to put a sign on it.
This action is not permitted.



In [84]:
put_sign(0, [0,1])

[['O' 'X' 'O']
 ['free' 'X' 'free']
 ['X' 'free' 'O']]


In [85]:
put_sign(1, [1,2])

[['O' 'X' 'O']
 ['free' 'X' 'O']
 ['X' 'free' 'O']]


In [86]:
gm.ended

True

In [87]:
put_sign(0, [2,1])

Broke rule 1: Nothing is allowed if the game is ended.
This action is not permitted.

